In [1]:
import chromadb
from rich.console import Console
from rich.table import Table
import ollama
import re

chroma_client = chromadb.PersistentClient(path="chroma_db_2")
collection = chroma_client.get_or_create_collection(name="doctor_profiles_2")
console = Console()

def sanitize_markup(text):
    return re.sub(r'\[.*?\]', '', text)

def fetch_relevant_specializations(patient_report):
    results = collection.query(query_texts=[patient_report], n_results=5)
    
    specializations = {metadata['Specialization'] for metadata in results.get('metadatas', [[]])[0] if 'Specialization' in metadata}
    return specializations

def recommend_doctors():
    console.print("\n[bold blue]Step 4: Enter Patient Report Details...[/bold blue]")
    symptoms = console.input("[bold green]Enter Symptoms:[/bold green] ").strip()
    history = console.input("[bold green]Enter Patient History:[/bold green] ").strip()
    diagnosis = console.input("[bold green]Enter Preliminary Diagnosis:[/bold green] ").strip()

    if not all([symptoms, history, diagnosis]):
        console.print("[bold red]Error: All fields are required.[/bold red]")
        return []

    patient_report = f"Symptoms: {symptoms}. History: {history}. Preliminary Diagnosis: {diagnosis}."
    relevant_specializations = fetch_relevant_specializations(patient_report)

    if not relevant_specializations:
        console.print("[bold red]No relevant specializations found.[/bold red]")
        return []

    console.print(f"\n[bold green]Retrieved {len(relevant_specializations)} relevant specializations: {', '.join(relevant_specializations)}[/bold green]")

    results = collection.query(query_texts=[patient_report], n_results=5)
    if not results.get('ids', [[]])[0]:
        console.print("[bold red]No relevant doctors found.[/bold red]")
        return []

    recommendations = []
    for i, metadata in enumerate(results.get('metadatas', [[]])[0]):
        name = metadata.get('Name', 'Unknown')
        specialization = metadata.get('Specialization', 'Unknown')
        experience = metadata.get('Years of Experience', 'N/A')
        distance = results.get('distances', [[float('inf')]])[0][i]

        if specialization.lower() not in {s.lower() for s in relevant_specializations}:
            console.print(f"[bold red]Skipping {name} ({specialization}) - Not a relevant specialization.[/bold red]")
            continue

        prompt = (
            f"Doctor: {name} - Specialization: {specialization} | Experience: {experience}\n"
            f"Patient Report: {patient_report}\nWhy is this doctor suitable?"
        )

        try:
            response = ollama.chat(model="MedLlama2", messages=[{"role": "user", "content": prompt}])
            generated_summary = response.get('message', {}).get('content', "No explanation provided.")
        except Exception as e:
            generated_summary = f"Error generating explanation: {str(e)}. This doctor is likely suitable based on specialization ({specialization}) and experience ({experience})."

        recommendations.append({
            "Name": name,
            "Specialization": specialization,
            "Experience": experience,
            "Distance": distance,
            "Why": generated_summary
        })

    return sorted(recommendations, key=lambda x: x['Distance'])[:5]

def display_recommendations(recommendations):
    if not recommendations:
        console.print("[bold red]No recommendations to display.[/bold red]")
        return

    console.print("\n[bold green]Step 6: Final Output[/bold green]")
    console.print("[bold blue]Recommendations:[/bold blue]")

    for i, rec in enumerate(recommendations, start=1):
        console.print(f"\n{i}. [bold cyan]{rec['Name']}[/bold cyan]")
        console.print(f"   - Specialization: [bold magenta]{rec['Specialization']}[/bold magenta]")
        console.print(f"   - Experience: [bold green]{rec['Experience']} years[/bold green]")
        console.print(f"   - Similarity Score: [bold yellow]{rec['Distance']}[/bold yellow]")
        console.print(f"   - Why: [bold white]{sanitize_markup(rec['Why'])}[/bold white]")

if __name__ == "__main__":
    display_recommendations(recommend_doctors())

Step 4: Enter Patient Report Details...

Enter Symptoms:

 Persistent cough, shortness of breath, chest pain.


Enter Patient History:

 Smoker for 20 years, family history of lung cancer.


...

5. Dr. Liam Brooks

- Specialization: Oncology

- Experience: 14 years

- Similarity Score: 1.5231346108250647

- Why: This oncologist has the right level of experience (14) to handle this case. She's also a specialist in 
her field and likely has experience with lung cancer cases. 